# ETL Entity and Topic Extraction

This notebook will aim to extract important topics from documents all documents. 
The output will be a cleaned dataset containing all text content for analysis.

In [1]:
import sys

sys.path.append(r'c:\users\owner\appdata\roaming\python\python37\site-packages')

import scispacy
from scispacy.abbreviation import AbbreviationDetector
import spacy
import pandas as pd
import json
import os
import re
from tqdm.notebook import tqdm
from rake_nltk import Rake
from spacy.matcher import Matcher 
import en_core_web_lg

from nltk.stem import WordNetLemmatizer 
import nltk
#nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()


nlp_1 = spacy.load('en_core_sci_lg')
nlp = spacy.load('en_core_sci_lg')
# Add the abbreviation pipe to the spacy pipeline.
abbreviation_pipe = AbbreviationDetector(nlp_1)
nlp.add_pipe(abbreviation_pipe)

## Identify and load file directories

In [2]:
directories = ['../data/biorxiv_medrxiv/', '../data/comm_use_subset/','../data/custom_license/', '../data/noncomm_use_subset/']

In [3]:
files = []
for directory in directories:
    for (dirpath, dirnames, filenames) in os.walk(directory):
        filenames = [names for names in filenames if '.json' in names]
        if filenames != []:
            files.append({'dirpath':dirpath, 'filenames':filenames})
        

### Initial string cleaning

**TODO: improving text cleaning will provide fundamental improvements.**

In [4]:
def clean(txt):
    txt=re.sub(r'\n',' ',txt)
    txt=re.sub(' +', ' ', txt)
    txt=re.sub(',', ' ', txt)
    txt=re.sub(r'\([^()]*\)','',txt)
    txt=re.sub(r'https?:\S+\sdoi','',txt)
    return txt.lower()

### Extract data classes from each document. 

**TODO: Additional infromation available expanding this could further enrich datasets**

In [5]:
def pub_extract(data):
    abstract = ''
    for section in data['abstract']:
        abstract = abstract + '  ' +  section['text']
    abstract = clean(abstract)
    
    text = ''
    for section in data['body_text']:
        text = text + '  ' + section['text']
    text = clean(text)
    ID = data['paper_id']
    title = data['metadata']['title']
    return ID, title, abstract, text


### Simple topic extraction can be used to cluster documents. 

In [6]:
def phrases_extract(text):
    r = Rake()
    r.extract_keywords_from_text(text)
    key_phrase = r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.
    
    return key_phrase

### Entity Extraction Functions

Inspired by: https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121

**TODO: Investigate the potential of biomedical specific NER model [1.] **

In [7]:
def filter_spans(spans):
    """Filter a sequence of spans and remove duplicates or overlaps. Useful for
    creating named entities (where one token can only be part of one entity) or
    when merging spans with `Retokenizer.merge`. When spans overlap, the (first)
    longest span is preferred over shorter spans.
    spans (iterable): The spans to filter.
    RETURNS (list): The filtered spans.
    """
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        # Check for end - 1 here because boundaries are inclusive
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
        seen_tokens.update(range(span.start, span.end))
    result = sorted(result, key=lambda span: span.start)
    return result

In [41]:
def refine_ent(ent, sent):
    unwanted_tokens = (
        'PRON',  # pronouns
        'PART',  # particle
        'DET',  # determiner
        'SCONJ',  # subordinating conjunction
        'PUNCT',  # punctuation
        'X',  # other
        )
    ent_type = ent.ent_type_  # get entity type
    if ent_type == '':
        ent_type = 'NOUN_CHUNK'
        ent = ' '.join(str(t.text) for t in
                nlp(str(ent)) if t.pos_
                not in unwanted_tokens and t.is_stop == False)
    elif ent_type in ('NOMINAL', 'CARDINAL', 'ORDINAL') and str(ent).find(' ') == -1:
        t = ''
        for i in range(len(sent) - ent.i):
            if ent.nbor(i).pos_ not in ('VERB', 'PUNCT'):
                t += ' ' + str(ent.nbor(i))
            else:
                ent = t.strip()
                break
    return ent, ent_type

In [107]:
def entity_pairs(text, coref=True):
    text = re.sub(r'\n+', '.', text)  # replace multiple newlines with period
    text = re.sub(r'\[\d+\]', ' ', text)  # remove reference numbers
    text = nlp(text)
    if coref:
        text = nlp(text._.coref_resolved)  # resolve coreference clusters
    sentences = [sent.string.strip() for sent in text.sents]  # split text into sentences
    ent_pairs = list()
    for sent in sentences:
        sent = nlp(sent)
        spans = list(sent.ents) + list(sent.noun_chunks)  # collect nodes
        spans = filter_spans(spans)
        with sent.retokenize() as retokenizer:
            [retokenizer.merge(span) for span in spans]
        dep = [token.dep_ for token in sent]
        try:
            if (dep.count('obj')+dep.count('dobj'))==1 \
                    and (dep.count('subj')+dep.count('nsubj'))==1:
                print('pass')
                for token in sent:
                    if token.dep_ in ('obj', 'dobj'):  # identify object nodes
                        subject = [w for w in token.head.lefts if w.dep_
                                   in ('subj', 'nsubj')]  # identify subject nodes
                        if subject:
                            subject = subject[0]
                            # identify relationship by root dependency
                            relation = [w for w in token.ancestors if w.dep_ == 'ROOT']  
                            if relation:
                                relation = relation[0]
                                # add adposition or particle to relationship
                                if relation.nbor(1).pos_ in ('ADP', 'PART'):  
                                    relation = ' '.join((str(relation),
                                            str(relation.nbor(1))))
                            else:
                                relation = 'unknown'
                            subject, subject_type = refine_ent(subject, sent)
                            token, object_type = refine_ent(token, sent)
                            ent_pairs.append([str(subject), str(relation), str(token),
                                    str(subject_type), str(object_type)])
        except:
            print('error')
    filtered_ent_pairs = [sublist for sublist in ent_pairs
                          if not any(str(x) == '' for x in sublist)]
    pairs = pd.DataFrame(filtered_ent_pairs, columns=['subject',
                         'relation', 'object', 'subject_type',
                         'object_type'])
    return pairs

## Keyword Search

Using keywords to link publication to Kaggle tasks.

In [59]:
with open('../tasks/keywords.json') as json_file:
    keywords = json.load(json_file)

def identify_tasks(text, keywords):
    text = lemmatizer.lemmatize(text)
    task_list=[]
    for k,v in keywords.items(): 
        if any(x in text for x in v['keywords']):
            task_list.append(k)  
    return task_list


## Apply each of the techniques outline above to each publication and aggrigate findings.

In [77]:
for file in tqdm(files):
    df = pd.DataFrame()
    directory = file['dirpath']
    for filenames in tqdm(file['filenames']):
        with open(directory + '/' + filenames, "r") as read_file:
            data = json.load(read_file)
        
        ID, title, abstract, text = pub_extract(data)
        try:
            key_phrases = phrases_extract(abstract)[0:5]
        except:
            key_phrases = []
            print('keyword error')
        try:
            abstract_tripples = entity_pairs(abstract, False).to_json()
        except:
            abstract_tripples = '{"subject":{},"relation":{},"object":{},"subject_type":{},"object_type":{}}'
            print('abs error')
        try:
            text_tripples = entity_pairs(text, False).to_json()
        except:
            text_tripples = '{"subject":{},"relation":{},"object":{},"subject_type":{},"object_type":{}}'
            print('keyword error')
        try:
            task_list = identify_tasks(text, keywords)
        except:
            task_list = []
            print('task list error')        
        
        df=df.append({'ID':ID,
                      'Repo':directory.split('/')[1],
                      'Title':title,
                      'Abstract':abstract,
                      'Text':text,
                      'Key Phrases':key_phrases,
                      'Abs Tri': abstract_tripples,
                      'Text Tri': text_tripples,
                      'Task List':task_list},ignore_index=True) 
        break
    break
    df.to_csv(directory +'CleanResults_1.2.csv',    index=False, header=False,)
    df.to_pickle(directory +'CleanResults_1.2.pickle')
        


[word count, 194 22 text word count]
[all rights]
[no reuse, permission]
[abstract, the positive stranded rna genomes, picornaviruses, a single large open reading 28 frame, 5′, 3′ untranslated regions]
[foot-and-mouth disease virus  29, 5′ utr  , five structural domains]
[the 30 internal ribosome entry site, which, initiation, translation  , the cis-acting 31 replication element]
[less well characterised structures, length, poly-c-tract, nucleotides, a series, tandemly, pseudoknots]
[we, investigated, the structures, pks 34, primer extension  analysis, their contribution, genome replication, mutation, deletion, experiments]
[36 shape, mutation, experiments, the importance, pk 37, function]
[deletion experiments, pks, replication  , they, genomes, a competitive advantage]
[39 replicons, full-length genomes, all pks, replication, rescued, genomes, less than one pk copy]
[consistent with, report, the presence, putative packaging signals, pk region]
[42 43 author/funder]
[all rights]
[no r

TODO: 
Update nlp model with:

[1] scispacy

@inproceedings{Neumann2019ScispaCyFA,
  title={ScispaCy: Fast and Robust Models for Biomedical Natural Language Processing},
  author={Mark Neumann and Daniel King and Iz Beltagy and Waleed Ammar},
  year={2019},
  Eprint={arXiv:1902.07669}
}

In [61]:
text_tripples

'{"subject":{},"relation":{},"object":{},"subject_type":{},"object_type":{}}'

In [62]:
abstract_tripples

'{"subject":{"0":"36 shape"},"relation":{"0":"confirmed"},"object":{"0":"importance"},"subject_type":{"0":"ENTITY"},"object_type":{"0":"NOUN_CHUNK"}}'

In [63]:
entity_pairs(text, False)

['vp3  and vp0  .', 'the p2 64 and p3 regions encode the non-structural proteins 2b and 2c and 3a  3b    3c pro and 4 structural protein-coding region is replaced by reporter genes  allow the study of genome 68 replication without the requirement for high containment  .', 'the fmdv 5′ utr is the largest known picornavirus utr  comprising approximately 1300 71 nucleotides and containing several highly structured regions.', 'the first 360 nucleotides at the 5′ 72 end are predicted to fold into a single large stem loop termed the s-fragment  followed by a the pks were originally predicted in 1987 and consist of two to four tandem repeats of a ~48 86 nucleotide region containing a small stem loop and downstream interaction site  87 . due to the sequence similarity between the pks   it is speculated that they 88 were formed by duplication events during viral replication  probably involving recombination.', '89 between two and four pks are present in different virus isolates but no strain ha

,subject,relation,object,subject_type,object_type


In [108]:
sS = """the fmdv 5′ utr is the largest known picornavirus utr comprising approximately 1300 71 nucleotides"""
entity_pairs(sS, False)

pass


,subject,relation,object,subject_type,object_type


In [109]:
sent = nlp(sS)
for ent in sent:
    print(ent.dep_, ent)

det the
compound fmdv
compound 5′
nsubj utr
cop is
det the
amod largest
amod known
compound picornavirus
ROOT utr
acl comprising
advmod approximately
compound 1300
nummod 71
dobj nucleotides


In [85]:
for c in sent.noun_chunks:
    print(c)

the fmdv 5′ utr
approximately 1300 71 nucleotides
several highly structured regions
